![cropped-SummerWorkshop_Header.png](../../resources/cropped-SummerWorkshop_Header.png)



<h1 align="center"> Neural Encoding </h1> 
<h2 align="center"> SWDB 2024 - Day 2 - Morning Session </h2> 
<h3 align="center"> Tuesday, August 20, 2024</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h2>Neural Encoding </h2>

Neural coding describes how neurons represent information about the world. Coding can be studied by asking whether external or internal events lead to changes in neural activity (<b>encoding</b>), or by asking whether different types of information can be read out from neural activity (<b>decoding</b>). In this workshop we will address the question of neural encoding, with a focus on single cell encoding models. 

Encoding of <b>sensory</b> information has been studied for decades by presenting animals with stimuli and observing how the activity of individual neurons changes. To study encoding of <b>motor</b> variables, researchers train animals to perform a behavior (or observe naturalistic behaviors) and correlate the animals' movement with changes in neural activity. Recent research has demonstrated that even sensory areas have representations of motor and behavioral variables, and vice versa. This is often called "multiplexed" coding. Furthermore, neural encoding can be influenced by <b>cognitive</b> processes such as learning, task engagement, and decision making. 

The exact form of neural activity changes is also a part of the study of neural coding. Neurons can represent information based on their average firing rates over a period of time, based on the precise spike times relative to some event, using bursts of spikes, or based on synchrony and timing relative to global population activity. In this workshop, we will consider <b>average firing rates</b> as they relate to information coding.

To learn more, check out this lecture Principles of Neural Coding by I. Memming Park: https://www.youtube.com/watch?v=DlFxUEdGlmQ
</div>


![neural_coding.png](../../resources/neural_coding.png)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p> 

Today we will look at how neurons in the visual cortex encode sensory and behavioral information in mice performing a visually guided task.

The first workshop of the day will focus on how single cells encode information in their average activity patterns (the problem of X-->R in the schemtic above). In the afternoon, we will learn about how to decode information from populations of neurons (R-->X as shown above) and how variability and correlations influence decoding.

## Workshop Outline & Questions

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h3> What we will investigate today </h3>

<h4> Part 1 -  Visual Behavior Ophys Dataset</h4>

(1) What is the experimental design? What are the key considerations for the question of neural encoding?

(2) What recording modality was used? Why choose this modality? What are the pros & cons?

(3) How can we access the data we are interested in?


<h4> Part 2 -  Tuning for stimulus & behavior during task performance </h4>

(1) Can we find neurons in the mouse visual cortex that are selective for specific visual stimuli? How reliable are their responses?

(2) Do stimulus responses differ depending on the mouse's behavioral choice during the task? 

(3) Do neurons in the mouse visual cortex modulate their activity as a function of running speed? 


<h4> Part 3 -  Quantifying single cell coding with regression models </h4>

(1) How can linear regression be used to model neural coding? 

(2) How do you ensure that your model is valid and is not overfitting?

(3) How well can you predict neural activity based on stimulus information? Behavioral information? 

(4) Does the prediction improve when additional variables are included? (multiple linear regression)



## Part 1 - Visual Behavior Ophys Dataset

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Experimental Design </h3>

(1) What is the experimental design? What are the key considerations for the question of neural encoding?
</p>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Stimulus / behavior paradigm: A visual change detection task </h4>

<p> To understand how neurons encode sensory and behavioral variables, it is useful to examine neural activity under conditions where sensory input and behavioral outputs vary across conditions and show rich and interesting structure that can be used to disentangle their unique contributions to neural signals. 

In the <b>Visual Behavior Ophys</b> dataset, neural activity was recorded while mice performed a visual change detection task. Mice were presented with stimuli and asked to make choices about those stimuli, while their <b>running</b> and <b>licking</b> behavior were measured, along with <b>pupil diameter</b> as a proxy for the animals' internal state. 

During the change detection task, mice view a continuous stream of <b>natural scene images</b>, which are displayed for 250ms, followed by a 500ms gray screen period. The gray screen period adds a working memory component to the task. The job of the mouse is to decide - "is what i am seeing now the same or different than what i saw 500ms ago?". 

If the mouse correctly detects a change and reports their choice by licking a reward spout within 750ms of the change, the trial is considered a <b>hit</b>, and a water <b>reward</b> is delivered. If mice fail to lick after a change, the trial is a <b>miss</b>. If the mouse licks anytime outside of the reward window just after the change (called a false alarm or aborted trial), the trial resets and the mouse will have to wait longer until the next opportunity for a reward. 

</div>

![change_detection_task.png](../../resources/change_detection_task.png)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Opportunities to study encoding of stimulus & behavior variables </h4>

<p> During the task, mice are free to run on a circular disc. Many studies have shown that locomotion can influence sensory signals and neurons can be tuned for running speed. 
Pupil diameter is also recorded, which can be used as a measure of overall behavioral state or arousal. When animals attend to a stimulus or are otherwise alert and active, the pupil dilates.
In contrast, during quiet or inattentive states, the pupil constricts. Neural coding is also influenced by arousal state, as measured by pupil diameter. 

Thus, in this dataset, we can ask about encoding of: 
* <b>Sensory stimuli</b> - via the images that are presented to the mouse
* <b> Behavioral choice</b>  - whether the mouse licks or not following a given stimulus presentation
* <b> Rewards</b>  - which are given depending on whether or not the mouse made a correct choice
* <b> Locomotion & arousal</b>  - via changes in animal running speed or pupil diameter

There may be additional dimensions of sensory or behavioral events that are of interest - can you think of any? 

Some other examples could be the number of exposures to a give image after a change, or the time since the last reward received by the mouse, or past trial outcomes, or perhaps a combination of running and pupil together is informative about cell activity.


</div>

![behavior_timeseries_color.png](../../resources/behavior_timeseries_color.png)

There are some interesting dynamics here - how might they influence neural activity?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Recording method </h3>

(2) What recording modality was used? Why choose this modality? What are the pros & cons?


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> 2-photon calcium imaging - pros & cons</h4>
<p>Today we will use the Allen Brain Observatory Visual Behavior Ophys dataset. 

"Ophys" stands for Optical Physiology, and typically refers to 2-photon calcium imaging of cellular activity (although there are other optical physiology methods!). You can learn more about this method in the *<b>DataBook</b>*

The critical thing to know for this workshop is that neural activity is measured as changes in the fluorescence of a genetically encoded calcium indicator, called GCaMP. Calcium influx occurs when neurons fire action potentials, thus the fluorescence of the calcium indicator is a proxy for neuronal spiking (however it is not a direct readout of neural spiking). 

Here are some additional considerations when choosing 2-photon calcium imaging as a recording modality:

Pros:
* Visualizing neurons in space
* Genetic targeting of specific neuron types
* Tracking neurons across multiple recording sessions

Cons:
* Slow acquisition rates relative to spike timing
* Limited to 500um depth, typically in cortex

</div>

Here is an example of what 2-photon calcium imaging looks like

<iframe width="560" height="315" src="https://www.youtube.com/embed/WGhMynyympg?si=panDVPhaaSTUAAy_" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Trangenic lines to label specific cell populations</h4>
<p>
In the <b>Visual Behavior Ophys</b> dataset, 3 different <b>transgenic mouse lines</b> were used to express GCaMP in either <b>excitatory neurons</b> (labeled by the Slc17a7-IRES2-Cre driver line), 
or in one of two types of <b>inhibitory neurons</b> - somatostatin (Sst) expressing neurons or vasoactive intestinal peptide (Vip) expressing neurons (Sst-IRES-Cre driver line and Vip-IRES-Cre driver lines, respectively). The reporter line driving GCaMP expression under the control of the Cre driver was either Ai93 or Ai148, both expressing GCaMP6f, which has fast kinetics. 

You can read more about transgenic mice in the <b>*DataBook*</b>

Sst and Vip inhibitory neurons are known to mutually inhibit each other and a shift in the balance between them can lead to disinhibition of excitatory neurons under certain conditions. Sst and Vip inhibitory neurons in the visual cortex, along with excitatory cells, are known to show modulation by locomotion, arousal, attention, and learning. 

Here is a useful review on how animal behavior and learning influence the coding of different cell types in the visual cortex: 
https://doi.org/10.1016/j.conb.2018.04.020

</div>

![cre_lines.png](../../resources/cre_lines.png)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

This dataset will allow us to ask not only about how neurons encode information, but also to ask which types of neurons encode which types of information. 
Combined with knowledge about anatomy and connectivity, this can help us understand how the brain computes information using circuits built up of unique cell types.

On day 3 of the workshop, you will learn about methods to map the morphology and synaptic connectivity of individual neurons, some of which are the same types of neurons recorded in this dataset. 

What kinds of questions can you address with 2-photon imaging alone? What kinds of questions could you address if you had both physiology and morphology? How can these different types of datasets inform each other?
 

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Multi-plane imaging across sensory & behavioral contexts</h4>

<p>
In in <b>Visual Behavior Ophys</b> dataset, a given population of neurons (i.e. a specific imaging plane) was measured across multiple sessions, and multiple imaging planes were recorded in each individual session. 

This experimental design allows analysis of changes in neural activity across days, under different <b>sensory and behavioral contexts</b>, and comparison of activity across different visual areas or cortical depths within a given session.

In some ophys sessions, mice perform the task with the image set they saw during training, which is highly <b>familiar</b>. 
In other sessions, mice perform the task with <b>novel</b> images they have never seen before. 

In addition to <b>active behavior</b> sessions where mice perform the task to earn rewards, there are also <b>passive viewing</b> sessions where the mice observe the stimulus in open loop, with no rewards delivered. In these passive sessions, mice are satiated and were given their daily water ration prior to the imaging session. 

</div>

![data_structure.png](../../resources/data_structure.png)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

These different features of the experiment means we need to keep track of a few things when doing analysis: 
<p>

* Which <b>session type</b> we are looking at (what image set was used? was it an active or passive session?)
  
* Which <b>brain area and cortical depth</b> the cells are from (i.e. which imaging plane it is within a session?)
  
* Which <b>genetically defined cell population</b> was imaged (i.e. what is the genotype of the mouse?)                                                                      

This information is provided as metadata by the <b>AllenSDK</b> toolkit, which you will learn how to use below.                                                                                                                                        

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

In the nomenclature of the <b>AllenSDK</b> we refer to each imaging plane within each session as an `ophys_experiment`.

The population of neurons in each imaging plane was tracked across multiple `ophys_sessions`, recorded on different days.

The collection of recording sessions belonging to a given imaging plane is called an `ophys_container`.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

For today's workshop, we will analyze an experiment from a transgenic mouse expressing GCaMP6f in 

<b>Sst neurons</b> in the 

<b>primary visual cortex</b> during an 

<b>active behavior</b> session with 

<b>familiar images</b>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Data access </h3>

(3) How can we access the data we are interested in?


In [1]:
# We need to import these modules to get started
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# seaborn makes pretty plots & sets font sizes nicely
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

# magic functions for jupyter notebook plotting
%load_ext autoreload
%autoreload 2
%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4>Using the AllenSDK toolkit</h4>

<p>
To identify experiments of interest based on the features of this dataset as described above, such as what cell populations were imaged, what types of sessions there were, etc., we need to access the metadata tables in the `VisualBehaviorOphysProjectCache` using the `AllenSDK` toolkit.

The `VisualBehaviorOphysProjectCache` class is responsible for downloading any requested data or metadata as needed and storing it in well known locations.  For this workshop, all of the data has been preloaded into data assets on CodeOcean - These data are big, and this will save us a lot of bandwidth and time.

</div>


In [2]:
# confirm that you are currently using the newest version of SDK (2.16.2)
import allensdk
allensdk.__version__

'2.16.2'

The code below shows you how to use the `VisualBehaviorOphysProjectCache` class to load metadata tables & explore the features of the dataset.

In [3]:
# This is the directory where files will be saved
# If using Code Ocean, this should link to the data directory, where the files will already be available
# output_dir = r'/scratch/'
output_dir = r'/Users/marinag/Documents/Data/visual_behavior_ophys_cache_dir'

In [4]:
# import behavior projet cache class from SDK to be able to load the data
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=output_dir)

/Users/marinag/anaconda3/envs/swdb_2024/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The cache contains methods that allow you to explore the types of recording sessions that exist in the dataset, and to load the data for individual experiments.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h3> Metadata tables </h3>


#### Load all cache tables

In [ ]:
# There are 4 metadata tables associated with the Visual Behavior Ophys dataset
behavior_session_table = cache.get_behavior_session_table()  
ophys_session_table = cache.get_ophys_session_table()   
ophys_experiment_table = cache.get_ophys_experiment_table()    
ophys_cells_table = cache.get_ophys_cells_table()                         


#print number of items in each table 
print('Number of behavior sessions = {}'.format(len(behavior_session_table)))
print('Number of ophys sessions = {}'.format(len(ophys_session_table)))
print('Number of ophys experiments = {}'.format(len(ophys_experiment_table)))
print('Number of unique cells = {}'.format(len(ophys_cells_table.cell_specimen_id.unique())))

What is the difference between the `ophys_session_table` and the `ophys_experiment_table`? 

In [ ]:
ophys_experiment_table.head()

In [ ]:
ophys_session_table.head()

The `ophys_experiment_table` contains one row for each imaging plane recorded in each session for all mice in the dataset. 

The `ophys_session_table` contains one row for each ophys session, which can contain one or more imaging planes. 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<p>
The metadata tables include a number of key details for understanding the dataset, such as - where the recordings were made, what type of cells were labeled, and what stimulus was shown in a given session. 

The *DataBook* describes all the columns of the metadata tables. We will explore a few of the most important ones here. 


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

First, let's narrow down our search and specifically look at the sessions for the <b>VisualBehaviorMultiscope</b> cohort. 

Filter the `ophys_session_table` to limit to the <b>VisualBehaviorMultiscope</b> `project_code` and assign the results to a new variable called `multiscope_sessions`.

How many mice are in this cohort? What mouse genotypes are available? (Hint: get the unique values of the `full_genotype` column)

In [ ]:
# Limit to a specific cohort / project code and check how many mice there are


In [ ]:
# Check what genotypes are available for this cohort


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h4>What is a genotype?</h4>
    
Typically, several transgenic lines of mice are bred together to create cell type specific expression of a gene of interest by combining a `driver line` (also called a `cre_line`) expressing Cre recombinase under the control of a specific gene of interest, and a `reporter line` that expresses some protein (such as GCaMP) under the control of Cre recombinase. This allows scientists to mix and match a variety of drivers & reporters to do different types of experiments. 

The `full_genotype`: describes the strategy that was used to label a given cell population with GCaMP.

The `cre_line`: is the first element of the `full_genotype` and determines which cell population is being targeted. In this dataset, it can be Slc17a7 for excitatory neurons, or Sst or Vip for different types of inhibitory neurons.

The `reporter_line`: is the final element of the `full_genotype` and determines what kind of reporter gene is expressed. In this dataset GCaMP6f is used.

You can learn more about transgenic mice and associated techniques in the <b>*DataBook*</b>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Let's look at an experiment where Sst inhibitory neurons were recorded. 

What are the unique values of the `mouse_id` column for mice with `cre_line` = <b>Sst-IRES-Cre</b> in the `multiscope_sessions` table we just made?

Pick the mouse with the largest value of `mouse_id` and assign it to a new variable called `special_mouse_id`.

What are the available values of the `session_type` column for this mouse?

Note the data type of the `mouse_id` column.

In [ ]:
# Filter by cre_line to get just the Sst mice, then print out the unique values of mouse_id, sorted in ascending order


In [ ]:
# Get the ID for the mouse with the largest number of mouse_id
# This is our special mouse
special_mouse_id = '546605'

In [ ]:
# Get all session types for our special mouse


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h4>What session type to choose?</h4>

The `session_type` column is a short hand description that conveys a several pieces of information about what the mouse experienced during a given session. Some of these pieces of information also have their own unique columns that you can search by. The `session_type` includes: 
<p>

* Whether the session was during <b>TRAINING</b> or <b>OPHYS</b> (the first element of the `session_type`)
* Which `image_set` was shown during that session (the second element of the `session_type`)
* The `behavior_type`, whether the session was <b>active behavior</b> or <b>passive viewing</b> (if the session type doesnt say `passive` at the end, that means it was an active behavior session)

<p>
Other columns that provide valuable information about what happened during a session include: 
<p>

`experience_level`: whether the session used <b>Familiar</b> or <b>Novel</b> images, and whether it was the first novel day (`Novel 1`) or a subsequent novel day `Novel >1`

`prior_exposures_to_image_set`: how many prior sessions the mouse has experienced with the image set being shown during the current session (should always be zero for `Novel 1` sessions)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Get the `ophys_session_id` for `session_type` = `OPHYS_1_images_A` for our special mouse. Save it to a variable called `familiar_session_id`.

What are the values of `experience_level` and `prior_exposures_to_image_set` for this session?

In [ ]:
# Get the session metadata for special mouse with the session type listed above
sessions = ophys_session_table[(ophys_session_table.mouse_id==special_mouse_id) & 
                                (ophys_session_table.session_type=='OPHYS_1_images_A')]
sessions.head()

In [ ]:
# Save the session ID to a variable called familiar_session_id
familiar_session_id = sessions.index.values[0]

In [ ]:
# What are the experience_level and prior exposures values for this session?


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h4>Where was imaging performed for this session?</h4>

Relevant metadata columns include: 

`imaging_depth`: Because Ca2+ imaging is an optical technique, recordings must be targeted to a specific focal depth of the microscope, corresponding to how deep in the tissue the images were collected. 
The values in the `imaging_depth` column indicate the distance from the cortical surface for each imaging plane that was recorded. 

`targeted_structure`: This is the brain area where the recording was made. 
In Allen Brain Observatory Ophys experiments, specific visual areas are targeted using Intrinsic Signal Imaging (ISI) to identify the boundaries of each visual area based on their reinotopic maps. You can learn more about this method in the <b>DataBook</b>.

As we saw previously, the `ophys_experiment_table` contains metadata for each individual image plane that was recorded in each session. Accordingly, information about which areas and depths were recorded can be found in the `ophys_experiment_table`, but not in the `ophys_session_table`. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the `ophys_experiment_table` to find all the imaging planes recorded in the `familiar_session_id` from our special mouse.

What `targeted_structures` were imaged? What are the available values of `imaging_depth`? What `equipment_name` was used to record this session?

In [ ]:
# Get all recordings for session type OPHYS_1_images_A for our special mouse using the ophys_session_id we saved above
experiments = ophys_experiment_table[ophys_experiment_table.ophys_session_id==familiar_session_id]
experiments.head()

In [ ]:
# Print out the targeted structures and the imaging depths


In [ ]:
# We can also look at both of these pieces of information at once by limiting the table just to those columns (plus a few others that might be interesting)
experiments[['targeted_structure', 'imaging_depth', 'session_type', 'experience_level', 'equipment_name']]

In this session, recordings were made in VISp and VISl, at multiple cortical depths. However we only see 4 imaging planes here (each represented by a unique `ophys_experiment_id`) - shouldnt we expect 8 imaging planes per session for multi-plane imaging experiments? 

While it is true that 8 imaging planes are recorded in each multi-plane imaging session (acquired using the `MESO.1` or `MESO.2` microscopes), there are strict quality control (QC) criteria applied to each imaging plane. 

Some of the 8 planes can fail QC while others pass. Examples of QC criteria include: how much brain motion there was for a given plane or whether the signal to noise was too low to detect cells.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Get the `ophys_experiment_id` for the recording in `VISp` at `225`um depth, and save it to a variable called `ophys_experiment_id`

In [ ]:
targeted_structure = 'VISp'
imaging_depth = 225
# Get the ophys_experiment_id for this area and depth for our specific session
ophys_experiment_id = experiments[(experiments.targeted_structure==targeted_structure) & 
                                    (experiments.imaging_depth==imaging_depth)].index.values[0]
print(ophys_experiment_id)


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h3> Physiology data </h3>

To load the data for a single imaging plane recorded in a given session, we can use the `get_behavior_ophys_experiment` method of the `VisualBehaviorOphysProjectCache` class that we instantiated previously as `cache`. 

This method returns a python object that contains all data and metadata for a given recording as attributes, along with some useful functions. We typically name this python object simply `dataset`.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the `ophys_experiment_id` we saved above as the input to the `get_behavior_ophys_experiment` method of the cache. 

Save the output to a variable called `dataset`. This is a python object that contains all the data for this imaging plane. 

Examine the `metadata` attribute.


In [ ]:
# Load the dataset for the ophys_experiment_id we selected 
dataset = cache.get_behavior_ophys_experiment(ophys_experiment_id)

In [ ]:
# Look at the metadata attribute


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h4>What physiology data is provided?</h4>
<p>

`dff_traces`: dataframe containing normalized fluorescence traces for each cell. dF/F or dFF stands for 'delta fluorescence over baseline fluorescence', i.e. the change in fluorescence relative to each cell's baseline signal. 

`events`: dataframe containing calcium events detected from fluorescence signals. events are detected based on the rapid rise in calcium, typically associated with bursts of spikes. Events have a time and a magnitude, roughly equivalent to the spike rate of a neuron.

`ophys_timestamps`: time, in seconds, of each imaging frame of the 2-photon movie. The indices of `dff_traces` and `events` correspond to the times in the `ophys_timestamps` array. Note that the frame rate of the recordings can vary, with 30Hz being typical for single-plane imaging sessions, and 11Hz typical for multi-plane imaging.

`max_projection`: array of maximum intensity projection image of the 2-photon movie. Allows visualization of pixels with large changes in fluorescence, corresponding to active neurons.  

`average_projection`: array of average intensity projection image of the 2-photon movie. Allows visualization of average fluorescence across the 2-photon field of view. 

`roi_masks`: dataframe containing regions of interest corresponding to neuron cell bodies, segmented from the 2-photon movies. Each cell trace comes from one of the roi_masks.

`segmentation_mask_image`: array containing all segmented ROIs.

`cell_specimen_table`: dataframe containing cell ROI information

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Plot the `max_projection` and `segmentation_mask_image` this imaging plane. How many ROIs are there?

In [ ]:
# plot the max projection image with plt.imshow()


In [ ]:
# plot the segmentation_mask_image


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Examine the `dff_traces` and `events` attributes. How are they formatted?


In [ ]:
# Look at dff_traces attribute


In [ ]:
# Look at events attribute


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

The ```events``` table is similar to ```dff_traces``` but the output provides traces of deconvolved events. Events are computed on spatially unmixed dff traces for each cell as described in [Giovannucci et al. 2019](https://pubmed.ncbi.nlm.nih.gov/30652683/). 

The magnitude of events approximates the firing rate of neurons with the resolusion of about 200 ms. The biggest advantage of using events over dff traces is they exclude prolonged Ca transients that may conteminate neural responses to subsequent stimuli. You can also use ```filtered_events``` which are events convolved with a filter created using ```stats.halfnorm``` method. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Pick one `cell_specimen_id` and plot dF/F and events for that cell, using `ophys_timestamps` for the y-axis values to show the time in seconds. 


In [ ]:
# Let's try the second cell (index one of the dff_traces and events tables)

# Get cell_specimen_ids from the cell_specimen_table. Can also get from either the dff_traces or events
cell_specimen_ids = dataset.dff_traces.index.values # a list of all cell ids
cell_specimen_id = cell_specimen_ids[1] # pick the second cell (index = 1 because python uses zero indexing)
print('Cell specimen id = {}'.format(cell_specimen_id)) # print the cell ID

In [ ]:
# Plot dff and events traces overlaid on the same axis for the cell selected above



We can see that as expected, events trace is much cleaner than dff and it generally follows big calcium transients really well.

This cell is particularly active towards the end of the session - whats up with that? 

If you've checked out the <b>*DataBook*</b> description of the Visual Behavior Ophys dataset, you would have seen that each Visual Behavior Ophys experiment has 10 repeats of a 30 second movie clip at the end of each session. 
It would be interesting to see how reliable the cell's response is across repeats of the movie. 

You can also learn about what stimuli are presented and when using the `stimulus_presentations` table.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h3> Stimulus presentations </h3>

Each ophys session is broken up into several <b>stimulus blocks</b>.

First, a 5 minute gray screen period occurs during which there are no visual stimuli. This is helpful to determine cells' baseline level of activity. There is another 5 minute gray screen period at the end of the session, followed by 10 repeats of a 30 second movie clip. 

The bulk of each ophys session is change detection task performance, which lasts for 60 minutes. 


![vbo_session_structure_2.png](../../resources/vbo_session_structure_2.png)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Examine the `stimulus_presentations` attribute of the dataset. What are the columns? 

What are the values of the `stimulus_block_name` column?

In [ ]:
# What does the stimulus_presentations table look like? 


In [ ]:
# What are the stimulus blocks?


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Assign the table to a variable called `stimulus_presentations`, so that we dont have to retrieve it from the dataset object every time we want to use it.

Select the `change_detection_behavior` block and look at the unique values of the `image_name` column for that block.

In [ ]:
# Assign the table to a variable
stimulus_presentations = dataset.stimulus_presentations.copy()

In [ ]:
# Limit to change detection behavior block


This table provides helpful information like image name, start, duration and stop of image presentation, and whether the image was omitted. 

In [ ]:
# What are the image names?


If you are curious what these images look like, you can check the <b>*DataBook*</b> to learn how to visualize them using the `stimulus_templates` attribute

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h4> Image changes and image omissions </h4>

You may have noticed that one of the values of `image_name` is "omitted". 
That is because some image presentations are randomly omitted during ophys sessions (but never during training).
This allows neural signals associated with the absence of an expected stimulus to be analyzed. 

The `omitted` column of the `stimulus_presentations` table also provides a useful Boolean value to filter by omissions. 

Another useful column is the `is_change` column, which is another Boolean value. 
This can be used to identify the image changes, which are the <b>go</b> trials of this task. 

You can also look at <b>no-go</b> or <b>catch</b>  trials using the `is_sham_change` column. 
This column is True for all image presentations that could have been a change, according to the exponential distribution of change times. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

How many unique `stimulus_presentations` are there in this session?

How many image changes were there? How many stimuli were omitted? 

In [ ]:
# Count all stimulus presentations
print(len(stimulus_presentations), 'stimulus presentations total')

# Count the changes
print(len(stimulus_presentations[stimulus_presentations.is_change==True]), 'stimulus presentations were changes')

# Count the omissions
print(len(stimulus_presentations[stimulus_presentations.omitted==True]), 'stimulus presentations were omitted')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h4> Timestamps </h4>

Now that we know how to get the stimuli for this session, we want to ask how neurons respond to different types of stimuli. 
This means we need to know when a given stimulus happened relative to the neural recordings. 

All the data in each session was recorded on a common clock, however not all data streams were sampled at the same rate. 
Let's examine the timestamps to understand the differences


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Examine the `stimulus_timestamps` attribute. Compare it to the values of `ophys_timestamps`. Are they the same? 

Compute the frame rate of each set of timestamps by using `np.diff` to get the inter-frame interval. The frame rate is 1 divided by the average inter-frame interval. 

In [ ]:
# Stimulus timestamps


In [ ]:
# Ophys timestamps


In [ ]:
# Stimulus frame rate
1/np.mean(np.diff(dataset.stimulus_timestamps))

In [ ]:
# Ophys frame rate
1/np.mean(np.diff(dataset.ophys_timestamps))

Note that the stimulus frames and ophys frames are acquired at different frame rates, however they are recorded on the same time clock.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
<h3> Getting stimulus aligned responses </h3>

As we saw above, the ophys data and stimulus presentations are not recorded at the same rate. 
If we want to compute stimulus aligned cell activity, we will need a way to associate ophys timestamps with the nearest stimulus timestamps. 

Fortunately, the `brain_observatory_utilities` package provides tools to make this easier. 

In [ ]:
import brain_observatory_utilities.datasets.optical_physiology.data_formatting as data_formatting

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

We can use the `get_stimulus_response_df` function from the `datasets.optical_physiology.data_formatting` module to get the stimulus locked activity for all cells in the dataset. 

Let's look at the documentation for this function


In [ ]:
# Let's look at the documentation for the get_stimulus_response_df function
data_formatting.get_stimulus_response_df?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

The `get_stimulus_response` function is smart and already knows what our data structures look like, 
so it can pull out the relevant information from the cell activity tables (`dff_traces` and `events`) 
and from the `stimulus_presentations` table.

The `ophys_experiment` argument to the `get_stimulus_response` function is an instance of the dataset object that we have been working with.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the `get_stimulus_response_df` function to get stimulus aligned dff traces for image changes, in a +/- 1s window around the change time.

What are the columns of the table that is returned?

In [ ]:
# Get stimulus aligned dff traces (using data_type input) for image changes (using event_type input)
# For a +/-1 second window (using time_window input)
# The default response_window_duration is 0.5, which means the average in a 0.5 second window after the stimulus onset will be computed
# interpolating the traces to 30Hz ensures that the timestamps are nicely consistent across trials, but this isnt strictly necessary
stimulus_response_df = data_formatting.get_stimulus_response_df(dataset, data_type='dff', event_type='changes',
                                                            time_window=[-1, 1], response_window_duration=0.5,
                                                            interpolate=True, output_sampling_rate=30)
stimulus_response_df.head()

In [ ]:
# What are the columns of your new stimulus response dataframe? 


Note that the stim_response_df contains the index of the `stimulus_presentations` table, the `stimulus_presentations_id`. 

This means that we can easily add stimulus information to the stimulus response dataframe by merging it with the `stimulus_presentations` table.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Add stimulus metadata by merging the `stimulus_response_df` and the `stimulus_table` using the `stimulus_presentations_id` column. 

Check the documentation for the `pandas.merge` function if you are unsure how to use it. https://pandas.pydata.org/docs/reference/api/pandas.merge.html 

In [ ]:
# Merge the stimulus_presentations table with the stimulus_response_df
stimulus_response_df = stimulus_response_df.merge(stimulus_presentations, on='stimulus_presentations_id')
stimulus_response_df.head()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

What columns exist in your new `stimulus_response_df`? 

What are the values of the `is_change` column (which we added via the merge function above)?

What are the values of the `trace_timestamps` column? Are they all the same? What are they referenced to? 

In [ ]:
# Check out the columns available


In [ ]:
# What are the values of the is_change column? Why are these the only values?


In [ ]:
# Whats up with the timestamps?


Looks like the timestamps go from -1 to 1 second around the stimulus onset time, just like we asked for when we ran the `get_stimulus_response_df` function!! 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Plot the average image change evoked response for the `cell_specimen_id` we selected earlier, using the `trace` and `trace_timestamps` columns.

In [ ]:
# Plot the stimulus aligned trace for the cell_specimen_id we care about



Looks like this cell reduces its activity after an image change. I wonder if thats a property of all Sst neurons, or just this one... 

What about other cell types like Vip inhibitory neurons or Slc17a7 excitatory neurons? Are they change modulated?

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Hurray!! We can exact stimulus evoked responses! Now we can finally start asking the questions we outlined at the beginning!! 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Part 2 -  Tuning for stimulus & behavior during task performance </h2>

(1) Are neurons in the mouse visual cortex selective for specific visual stimuli? How reliable are their responses?

(2) Do stimulus responses differ depending on the mouse's behavioral choice during the task? 

(3) Do neurons in the mouse visual cortex modulate their activity as a function of running speed? 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

To understand sensory tuning, we want to know how individual cells respond to different stimuli. We will test this out first.

To understand the impact of behavior on single cell coding, we could take a few different approaches. 

One aspect of <b>behavior</b>  is the animals behavioral <b>choice</b> during the task. 
We could ask whether stimulus tuning, or overall cell activity, is different for image changes when the mouse correctly responds to the change with a lick and got a reward(i.e. a <b>hit</b> trial), compared with changes where the mouse failed to respond (i.e. <b>miss</b> trials)

Another aspect of <b>behavior</b> is the animals overall behavioral state, such as whether they are <b>running</b> or <b>stationary</b>. 
Here we could ask whether a given cell's activity level is modulated by the overall speed of the mouse (i.e. are cells "tuned" for running speed?). Alternatively, we could look at how cell activity varies with <b>pupil diameter</b>, which is a measure of arousal and is correlated with various measures of brain state. 


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h3> Stimulus tuning & response variability </h3>

Let's start by asking whether individual cells respond differently to the 8 different images shown in each Visual Behavior Ophys session

Then we will evaluate how consistent that response is, and whether it is valid to claim that the cell "encodes" a given image. 

Let's revisit the `stimulus_response_df` for the `cell_specimen_id` we are interested in.

The `mean_response` column contains the average value of the dF/F signal (which is what we provied as the `data_type` to the `get_stimulus_response_df` function - we could replace that with `events` to use deconvolved events instead) in a pre-defined window of time following the stimulus onset (determined by the value of `response_window_duration` provided to the `get_stimulus_response_df` function above). 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Get the data from `stimulus_response_df` just for our `cell_specimen_id` of interest and assign it to its own variable.

Get the average value of the `mean_response` column for each unique `image_name` in the `stimulus_response_df` for our cell and plot it. 

The y-axis should be the value of the `mean_response` and the x-axis should be the `image_name`. 

Bonus points for using `pandas.groupby` for this: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html 

In [ ]:
# Get data for our cell, which we should have saved to a variable called cell_specimen_id above
cell_df = stimulus_response_df[stimulus_response_df.cell_specimen_id==cell_specimen_id]
cell_df.head()

In [ ]:
# Get average of mean response column for each image
# You could do this using a for loop, but using pandas groupby is better



In [ ]:
# Plot the mean response for each image 
# Make sure to label your axes!



<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now plot the `mean_response` for each individual presentation of each image, along with the average response across all repetitions of each image as we did above. 

How variable is the cell activity across repeated presentations of a given image?

Bonus points for using `seaborn.scatterplot` function with the `stimulus_response_df` dataframe for this cell as the input: https://seaborn.pydata.org/generated/seaborn.scatterplot.html


In [ ]:
# Plot the mean response for each image

# Get cell data and compute tuning curve using groupby
cell_df = stimulus_response_df[stimulus_response_df.cell_specimen_id==cell_specimen_id]
tuning_curve = cell_df.groupby(['image_name']).mean()[['mean_response']]

# Get sorted image names for x-axis
image_names = np.sort(tuning_curve.index.values)

# Make the plot
fig, ax = plt.subplots(figsize=(6,4))
ax = sns.scatterplot(data=cell_df, x='image_name', y='mean_response', ax=ax)
ax = sns.pointplot(data=cell_df, x='image_name', y='mean_response', order=image_names, color='r', linestyle='--', ax=ax)
ax.set_title('cell_specimen_id: '+str(cell_specimen_id)+'\nImage selectivity')
ax.set_xticklabels(image_names, rotation=90)
ax.set_ylabel('dF/F')

What could account for the trial to trial variability here? 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

What do other cells' tuning curves look like? 

Get the average image response for all cells in this experiment and plot it as a heatmap. 

Super mega bonus points if you use `pandas.groupby`, `pandas.pivot_table`, AND `seaborn.heatmap` for this

https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
# Get the average response across all presentations of each image for each cell in this session using pandas groupby


In [ ]:
# Use pivot_table to convert it into a matrix of cells by images


In [ ]:
# Now plot that matrix using seaborn's heatmap function


Does this pattern reflect the true selectivity of these cells, or is it just random chance? 

One way to test thais is to ask whether image tuning differs depending on which trials you select

Let's try splitting the data, then plotting the same heatmap for the splits. Will it look the same?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Splitting the data </h4>

Let's try splitting the data and checking if the tuning curves look the same when computed separately for the first and second halves of the session

In [ ]:
# Get the total number of stimulus presentations and get the halfway point in the session
stimulus_presentation_ids = stimulus_response_df.stimulus_presentations_id.unique()
print('there are', len(stimulus_presentation_ids), 'stimulus presentations in this dataframe')
print('the halfway point is ~ trial', int(len(stimulus_presentation_ids)/2))

In [ ]:
# Split the data into first and second half using this threshold on stimulus_presentations_id
first_half = stimulus_response_df[stimulus_response_df.stimulus_presentations_id.isin(np.arange(0, 156))]
second_half = stimulus_response_df[stimulus_response_df.stimulus_presentations_id.isin(np.arange(156, len(stimulus_presentation_ids)))]

In [ ]:
# Compute tuning curves for the first half, using groupby and pivot_table as we did above
tuning_curves_first_half = first_half.groupby(['cell_specimen_id', 'image_name']).mean()[['mean_response']]
tuning_curves_first_half = tuning_curves_first_half.pivot_table(values='mean_response', index='cell_specimen_id', columns='image_name')

# Compute tuning curves for the second half
tuning_curves_second_half = second_half.groupby(['cell_specimen_id', 'image_name']).mean()[['mean_response']]
tuning_curves_second_half = tuning_curves_second_half.pivot_table(values='mean_response', index='cell_specimen_id', columns='image_name')

# Plot the heatmap for the tuning curves computed on each split of the data
fig, ax = plt.subplots(1, 2, figsize=(10,4), sharey=True)
ax[0] = sns.heatmap(data=tuning_curves_first_half, vmax=0.5, cbar=True, ax=ax[0])
ax[0].set_title('first half of trials')
ax[1] = sns.heatmap(data=tuning_curves_second_half, vmax=0.5, cbar=True, ax=ax[1])
ax[1].set_title('second half of trials')

These look pretty different!! What could cause these differences? 

One factor could be task engagement. Mice often perform the task better in the first half than the second half, because they are more motivated. 
When they are engaged, most of the trials are hits. When the mice disengage, they have a lot more misses.

We can split the data by hit and miss trials to see whether the mouse's behavioral choice influences tuning. 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h3> Does behavior choice affect stimulus response? </h3>

Let's try splitting the data based on whether each image change resulted in a <b>hit</b> or a <b>miss</b> and see if the mouse's behavioral choice influences the response of our cell of interest



![Trial_diagram.png](../../resources/Trial_diagram.png)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Above we looked at responses to individual images that were shown during the session. 
Now we want to look at image changes that were either a <b>hit</b>, where the mouse correctly licked following the change, or a <b>miss</b>, where the mouse failed to lick after the change. 

First we need to figure out whether the mouse correctly licked following each image change or not

We could do this by determining whether there was a lick or a reward for each trial. Let's look at the `licks` and `rewards` attributes of the dataset object

In [ ]:
# Examine the licks attribute of the dataset


In [ ]:
# Examine the rewards attribute


To figure out which image changes had a correct lick or not (and thus correspond ot a hit or a miss), we would need to compare the onset times of each image change in the `stimulus_presentations` table to the lick times in the `licks` table (or the reward times in the `rewards` table) to see if there was a lick (or a reward) within 750ms of the stimulus onset. 

This is technically straightforward but can be tedious, so to save some time so that you can focus on asking interesting questions rather than data munging, we will provide you with some tools to annotate the `stimulus_presentations` table with things like lick and reward times for each image presentation. 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h4> Annotating stimulus presentations with behavior information </h4>
<p>

The `brain_observatory_utilities` package provides a useful tool to annotate the `stimulus_presentations` table with information about what happened during each stimulus, including timing of `licks`, `rewards`, and whether the trial was a <b>hit</b> or a <b>miss</b> trial. 

It will also add the average `running_speed` and `pupil_width` for each stimulus presentation. These can be used to filter data, or plot directly against cell activity to ask about he relationship between running and neural activity. 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

The `get_annotated_stimulus_presentations` function can be found in the `datasets.behavior.data_formatting` module of `brain_observatory_utilities`. It takes in the <b>dataset</b> object, which contains everything it needs to know about stimulus presentations, licks, rewards, running, etc., and returns an annotated version of the `stimulus_presentations` table.

Let's check out the documentation

In [ ]:
import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_utils

In [ ]:
behavior_utils.get_annotated_stimulus_presentations?

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the `get_annotated_stimulus_presentations` function to get the annotated stimulus presentations table and assign it to a variable called `annotated_stim_table`. Inspect the columns.

In [ ]:
# Provide dataset object to the get_annotated_stimulus_presentations function and assign the results to a new variable
annotated_stim_table = behavior_utils.get_annotated_stimulus_presentations(dataset)

In [ ]:
# Look at the result


In [ ]:
# Look at all the useful new columns!


To be able to sort cell activity based on whether each image change in the stimulus table was a hit or a miss, we will want to merge the `stimulus_response_df` and with this new annotated table. Let's recompute the stimulus response dataframe and merge it with the `annotated_stim_table`.

In [ ]:
# Get the stimulus response dataframe just for image changes
stimulus_response_df = data_formatting.get_stimulus_response_df(dataset, data_type='dff', event_type='changes',
                                                            time_window=[-1, 1], response_window_duration=0.5,
                                                            interpolate=True, output_sampling_rate=None)

# Merge it with the annotated stim table so you can filter cell responses based on behavior choice                                                       
stimulus_response_df = stimulus_response_df.merge(annotated_stim_table, on='stimulus_presentations_id')


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>
Now we can plot tuning curves separately for hits and misses.

Limit the `stimulus_response_df` to image changes, then split by hit & miss trials. 

Plot our cell's image tuning curve (and the variability around the mean) using seaborn's pointplot, showing hits and misses using different colors

In [ ]:
# Get responses just for our cell, then split into hit and miss trials
cell_df = stimulus_response_df[stimulus_response_df.cell_specimen_id==cell_specimen_id]
hits = cell_df[(cell_df.is_change)&(cell_df.hit)]
misses = cell_df[(cell_df.is_change)&(cell_df.miss)]

In [ ]:
# Plot the mean response for each image split by hit and miss

# Make sure that the x-axis is sorted by image name
image_names = np.sort(cell_df.image_name.unique())

# Plot hits and misses
fig, ax = plt.subplots(figsize=(7,4))
ax = sns.pointplot(data=hits, x='image_name', y='mean_response', order=image_names, color='r', linestyle='--', ax=ax)
ax = sns.pointplot(data=misses, x='image_name', y='mean_response', order=image_names, color='b', linestyle='--', ax=ax)
ax.set_title('cell_specimen_id: '+str(cell_specimen_id)+'\nImage selectivity')
ax.set_xticklabels(image_names, rotation=90)
ax.set_ylabel('dF/F')
ax.legend(['hits', 'misses'], loc='upper left')

In [ ]:
# You could plot the same thing using the 'hue' input to seaborn pointplot
fig, ax = plt.subplots(figsize=(7,4))
ax = sns.pointplot(data=cell_df[cell_df.is_change], x='image_name', y='mean_response', hue='hit', order=image_names, linestyle='--', ax=ax)
ax.set_title('cell_specimen_id: '+str(cell_specimen_id)+'\nImage selectivity')
ax.set_xticklabels(image_names, rotation=90)
ax.set_ylabel('dF/F')
ax.legend(title='Hit')

What if it isnt actually the mouse's choice thats influencing the differences in activity seen here? 

What if the mouse runs more or less in the first and second half of the experiment, or if the pupil diameter is correlated with whether or not the mouse receives a reward or not?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h3> Are cells tuned for running speed or pupil diameter? </h3>

Another factor that could influence neural variability and contribute to neural encoding is locomotor behavior, or the behavioral state of the animal. Many studies have shown that animal movement and overall arousal state can influence the gain of sensory tuning. Running and other movements are also directly encoded by some neurons in the visual cortex, independent of stimulus identity. 

The dataset object contains info about the mouse's `running_speed`, in addition to information about pupil diameter and gaze location in the `eye_tracking` attribute. Running speed and pupil diameter are typically correlated, and both can be used as measures of overall arousal and behavioral state. 

Let's plot the activity of our Sst cells as a function of the mouse's running speed or pupil diameter to see if these neurons encode behavioral variables.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Get the `running_speed` attribute of the dataset object and plot the running speed. 

Remember that running speed is sampled at the stimulus display frequency, so you can use `stimulus_timestamps` to plot time on the x-axis. 

In [ ]:
# Plot the running speed, with stimulus_timestamps on x-axis
fig, ax = plt.subplots(1,1, figsize = (20,5))



<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Examine the `eye_tracking` attribute of the dataset object. What are the columns? 

Plot `pupil_area` over time. 

In [ ]:
# Whats in the eye_tracking table?



In [ ]:
# Plot the pupil area over time
fig, ax = plt.subplots(1,1, figsize = (20,5))



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

A note about filtering and signal processing

Note that there are some very large spikes in the pupil area in some parts of the session. These are probably artifacts of the pupil detection algorithm, and could be filtered out using `scipy.signal.medfilt`: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.medfilt.html. This function works by setting each point to be the median of its immediate neighborhood of `kernel_size` points, so is a good tool for data with obvious outliers. 

Note that because kernel size is measured in points rather than timebins, this function makes the most sense to use when sampling rates are fairly consistent.

In [ ]:
from scipy.signal import medfilt

fig, ax = plt.subplots(1,1, figsize = (20,5))
fit_pupil_area = medfilt(dataset.eye_tracking.pupil_area, kernel_size=21)   
ax.plot(dataset.eye_tracking.timestamps, dataset.eye_tracking.pupil_area, color='magenta')
ax.plot(dataset.eye_tracking.timestamps, fit_pupil_area, color='gray')

ax.set_xlabel('Time in session (seconds)')
ax.set_ylabel('Pupil area (pixels^2)')
ax.set_title('Ophys experiment {}'.format(ophys_experiment_id), fontsize = 20)

Now that we have a handle on this behavior data, lets try plotting running speed and tuning together. Do they look like they might be related to one another?

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Plot the `running_speed` and `pupil_area` on the same axies

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20,5))
fit_pupil_area = medfilt(dataset.eye_tracking.pupil_area, kernel_size=21)   
ax.plot(dataset.stimulus_timestamps, dataset.running_speed['speed'], color='orange')

ax2 = ax.twinx()
ax2.plot(dataset.eye_tracking.timestamps, fit_pupil_area, color='magenta')

ax.set_xlabel('Time in session (seconds)')
ax.set_ylabel('Pupil area (pixels^2)')
ax.set_title('Ophys experiment {}'.format(ophys_experiment_id), fontsize = 20)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Tuning for continuous variables </h4>

One challenge in working with these data is that running, eye tacking, and neural activity are all sampled on separate data streams with different timestamps. This means that even though these data were all collected at the same time, there isn't necessarily a one-to-one matchup between timestamps in one data stream or other.

The most common solution to this solution to this problem is data resampling. Typically timestamp bins are defined, and data are resampled into a common time stream. What size bin should you use? This depends on the timescale that is relevant to the analysis at hand.

For today, we will be using stimulus-presentation bins to look at our data over a relatively large timescale. Specifically, we will use the `stimulus_response_df` that we generated above, which contains the mean response for each stimulus presentation, along with the `annotated_stimulus_presentations` table that we merged into it, which contains the mean running speed for each stimulus presentation. 


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Get the stimulus response data for our special cell and plot `running_speed` and `mean_response` against each other

In [ ]:
# Get the stimulus response dataframe just for a particular cell
cell_df = stimulus_response_df[stimulus_response_df.cell_specimen_id==cell_specimen_id]

Now that we have data binned by stimulus presentation, lets try plotting the relationship between running speed and the activity of our cell.  


In [ ]:
# Plot this cells mean response versus running speed for each stimulus presentation


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Repeat using the `mean_pupil_width` column.

In [ ]:
# Plot mean pupil width across trials against this cells mean response


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>


Great! Our encodes both pupil diameter and running speed. 

You will recall, however, that these variables themselves are also correlated looked like they might have had a relationship to each other. Now that we have nicely binned data, try explicitly plotting the relationship between `mean_pupil_width` and `mean_running_speed`

In [ ]:
# Plot pupil width and running speed binned by stimulus presentations against each other


So...which of the factors best explains the variability in trial-to-trial responses of our cell? 

In the next section, we will dive deeper into this problem. Specifically, we will use regression models to pull apart the variance explained by different concurrent features on their own, as well as their interactions.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Part 3 -  Quantifying single cell coding with regression models </h2>

Up to now, we looked at how single cell activity varies across different conditions, like which image was shown, or whether the mouse was running or not. 
But are these conditions reliable predictors of cell activity? 

To say that a cell "encodes" something, we want to know that the cell activity is reliably predictable based on that something. 
In other words, can we model a cell's activity based on different variables or predictors?

Regression models provide a mathematical framework for investigating these questions. Here, we will use linear regression to investigate which behavioral features are encoded by neurons.


Questions: 
    
(1) How can linear regression be used to model neural coding? 

(2) How do you ensure that your model is valid and is not overfitting?

(3) How well can you predict neural activity based on stimulus information? Behavioral information? 

(4) Does the prediction improve when additional variables are included? 



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h3> Linear regression </h3>

    
In a regression problem, we are have pairs of data points  $(𝑥⃗_𝑖,𝑦_𝑖)$
  where  𝑖∈[1,𝑁]. We want to develop a function  $𝑓(𝑥⃗ )$
  such that  $𝑓(𝑥⃗_𝑖)≈𝑦_𝑖$
  for each pair of points in the data set.
    
    
The simplest regression problem is linear regression, in which we try to create the function $f$ by linearly combining a set of functions that act on the points $x$.

$f(\vec{x}_i) = \sum_j w_j \phi(\vec{x}_i)$

The functions $\phi(\vec{x})$ are chosen according to the question you are trying to answer. They are often called "features".  
    
The coefficients $w_j$ are called "weights." When we talk about fitting a regression model, what we mean is determining the best set of weights for our  $𝑓(𝑥⃗_𝑖) \rightarrow 𝑦_𝑖$ mapping? 



But what is the "best" set of weights? We try to choose the weights that minimize overall error between $f(x)$ and $y$.In the case of linear regression we use the sum of squared residuals between our for each $𝑓(𝑥⃗ 𝑖)$ and the corresponding $y_i$:

$E = \frac{1}{2} \sum_i \left | y_i - f\left ( \vec{x}_i \right ) \right |^2 = \frac{1}{2} \sum_i \left | y_i - \sum_j w_j \phi (\vec{x}_i ) \right |^2 $



This particular problem has an exact analytic solution that is easy to implement, but in this tutorial, we will look at how to perform regression using the `scikit-learn` Python package.  `scikit-learn` has many regression algorithms in common use built in, most of which do not have simple analytic solutions.  In addition, other packages have adopted the `scikit-learn` style interface.  One advantage of this is that multiple algorithms can be deployed with the same code.



The `scikit-learn` website:  http://scikit-learn.org/stable/

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Question 1: How can linear regression be used to model neural coding? </h3>
<p>

You may be familiar with a version of linear regression where the functions  𝜙 are chosen to be the identity and a constant. When the input space is one dimensional this is:

$𝑓(𝑥_i)=\sum_j w_j \phi(\vec{x}_i) = 𝑤𝑥_i+c$

This simple model assumes that $f(x)$ scales linearly as a function of $x$. 

However, even if the variables in our model do not have a perfect linear relationship, this model might still be useful; in practice, so long as $x$ and $y$ have a monotonic relationship, we would still expect to see the model explain some fraction of the variance in our data. This is equivalent to saying that $x$ and $y$ are linearly correlated.

Above, we noted that some cells have a correlation with the animals pupil width during each stimulus. First, lets use linear regression to mathamatically formalize this relationship. 

Once again, we can use the `stimulus_response_df` we computed to get the mean pupil width and mean response for each stimulus presentation. We will use these to start fiting our model. This time around, however, we will our pupil width `X` and our mean response `y` to be consistent with the math we just saw.

In [ ]:
# To match the equations above and the sklearn convention,
# We will call our predictor / encoded variable "X" and our response variable "y"
X = cell_df.mean_pupil_width.values
y = cell_df.mean_response.values
fig, ax = plt.subplots()
ax.plot(X, y, '.')
ax.set_xlabel('Mean pupil width')
ax.set_ylabel('Mean cell response')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

If $x$ and $y$ are correlated, we will fit a non-zero value for $w$ when we do our model fit. However, simply noting the correlation would be *Descriptive* - it wouldn't tell us anything about how consistent this correlation is across the dataset. A few spurious data points could lead to a correlation that does not hold throughout our data. 

`scipy` has pre-implemented calculators for data correlation. Here, we can quickly test if, for example, our data are pearson correlated.

In [ ]:
from scipy.stats import pearsonr

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use `pearsonr` to compute the correlation between the mean cell response and the mean pupil width on each stimulus presentation.

In [ ]:
# Compute the correlation of X and y (pupil width and cell activity)
pearson_corr,pearson_pval = pearsonr(X, y)
pearson_corr

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Regression provides a *Predictive* model. For new values of $x$, we can produce an estimate of what $y$ should be. Importantly, the predictive nature of our model also proves to be an important tool for assessing whether our model consistently represents our data. 

We do this by splitting our dataset into parts. Just as we did above, we will train the model on on part of our dataset, then evaluate it on data that was withheld from this initial training.

In [ ]:
# Split the data 
from sklearn.model_selection import train_test_split

np.random.seed(4)# Setting the random seed here insures that everyone gets the same result when they run this notebook!

# Use sklearn train_test_split 
y_train, y_test, X_train, X_test = train_test_split(y, X)

In [ ]:
# What do each of these splits look like? 
print('length of y_train', len(y_train))
print('length of y_test', len(y_test))
print('length of X_train', len(X_train))
print('length of X_test', len(X_test))


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Now that we have our data ready, we can import the `scikit-learn` package (we will call it "sklearn" to save some typing). It has a nice interface for fitting regression models that allows us to not worry about implementing our own solution for the cost function above.

In [ ]:
from sklearn.linear_model import LinearRegression

The sklearn interface is object oriented. This means that to fit a model, we need to instantiate a "LinearRegression" object. We will then use this for to handle our fitting.

In [ ]:
# Initialize the LinearRegression object
LR = LinearRegression()

Now that we have our object, we can fit data using the built in "fit" function.

In [ ]:
# Note that LinearRegression requires X to be two dimensional - why this is will be apparent shortly
LR.fit(X_train.reshape(-1, 1), y_train)

<!-- Thats it! We have our first model!  -->

We can now look carefully at the `sklearn` object to learn about the model fit we just performed. Here, `coef_` contains the weight vector $\vec{w}$ for our model, and `intercept_` contains the constant $c$

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Examine the `coef_` and `intercept_` attributes of the `LinearRegression` object.

In [ ]:
# Print the coef_ attribute of the LinearRegression object


In [ ]:
# print the intercept_ attribute of the LinearRegression object


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now that we know the weight and intercept for this model, plot the line we just fit, overlaid with the data.

In [ ]:
# Plot x and y values
fig,ax = plt.subplots()
ax.plot(X_train, y_train,'.')
ax.plot(X_test, y_test,'.')
# Plot the fit 
xx = [np.min(X),np.max(X)]
ax.plot(xx,LR.coef_*xx+LR.intercept_)
# Label the axes
ax.set_xlabel('Mean pupil width')
ax.set_ylabel('Mean cell response')
ax.set_title('Corr. coeff. = {:.2}'.format(pearson_corr));

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Now let's look at how well our model does at predicting data.

The `LinearRegression` object has a method to evaluate the `score` of the model. 

Here we will provide the training data for X and y (pupil and cell response)

In [ ]:
# How did we do with our training data.
LR.score(X_train.reshape(-1,1), y_train)

Is this number meaningful? We will delve into this now.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Question 2: How do you ensure that your model is valid and is not "overfitting"?</h3>
<p>

"Overfitting" is a term used to describe the case in which learns does very well in describing the data that it is trained on, but fails to predict new or additional data. Another way of saying this that a model will learn to describe noise or idiosyncrasies of the training data, rather than the underlying relationships that you are trying to model.

To illustrate overfitting, lets pause for a quick thought experiment. Imagine that, instead of fitting the two parameter model we just used, we fit a model with $N$ parameters where $N$ is the number of data points. Our model, which could look like this:

$𝑓(𝑥_i)=\sum_{i = [1,N]} w_i\vec{x}_i$

We call this a "saturated model" because it is saturated with parameters. Once we fit this model, we would discover that we could now *perfectly* predict every single data point. In this linear case, we would now find $f(x_i)=y_i$, with error of 0. 

So...why don't we do this? Wasn't our goal to get the lowest error possible? 

You have probably already noticed the two big problems with this saturated model. First, we can't learn anything from its weights. Regression is guided dimensionality reduction exercise, where we try and describe our data with a chosen set of features. The saturated model fails to do this. Second, this model is worthless for explaining new data. It assumes a 1-to-1 $x_i\rightarrow y_i$ mapping, and is undefined for new points or allows assumes no variance for repeated observations.

Even if we move away from this extreme case, it is still possible to overfit a model simply by fitting weights to too many features. Spurious correlations in your data will appear to be explained by the additional features in your training data, only to limit your ability to predict held out data. 

<b>
In this section, we will introduce a technique known as "Cross-Validation" as a way to systematically test if you have a "good" model.

<p>


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

*One more aside about saturated models: the deliberately overfit saturated model is always going to be the model with the lower error possible error. It is therefore useful as as a comparison tool to determine how well your model fit does. In the linear case, the error of the saturated model is always 0 so we don't really need to think about it. If, however, we are doing non-linear regression, the saturated model becomes a useful in assessing model performance.*

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

What is the models performance on our held-out testing data (`X_test` and `y_test`?

Use the `score` method again, but provide the held out test data instead.

Is it lower or higher than the score for the training data? Is it greater than or less than zero? What can we learn from answering these questions?


In [ ]:
# Compute the score for the held out test data



A score of 0.028 indicates that we explain 2.8% of the variance in our test data by using this model. This isn't a lot, and it is lower than the variance we explained using the training data suggesting that we aren't doing quite as well at predicting the pupil-neuron relationship in this new data. However, it is greater than 0, which means our model has SOME predictive power, even if it is small!

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

This "training" and "testing" split approach would be great if data were always cheap and plentiful. In practice however, it can be frustrating to use most of your hard-earned data to train a model, only learn how it performs on a held out subset.

A common aproach to dealing with this problem is known as <b>*Cross Validation*</b> Here, we systematically hold out chunks of data, refitting our model on the remaining data each time. By performing multiple model fits, we can (1) use all our data and (2) get a better sense of how our data varies across the dataset.

There are many ways to do cross validation, and how you split things up can have a big influence on the question you are trying to answer. Lets start with one of the simplist and most common forms of cross validation, known as <b>KFolds</b>. Here, we split (i.e. fold) our data $k$ times, with equal sample sizes in each fold. We then fit $k$ models to our data. 

What is <b>$k$</b>? We will use 5 for now. It can be anywere from 2 to $n$ where $n$ is the number of samples in your dataset. This extreme case, where $n$ is the same as the number of samples you have is called "leave-one-out" cross validation.



Here is an example of how to do 5 fold cross validation

In [ ]:
# Simple 5 fold cross validation
n_folds = 5
fold_num = np.zeros(len(X))

# Sort data into 5 groups:
fold_group = np.arange(0,len(X))%n_folds
fold_group

Now that we have groups, we can loop through and fit a model to each.

In [ ]:
# Compute the test & training score for each fold

# Create an empty array to save the data in
self_score = np.empty(n_folds)
cross_score = np.empty(n_folds)
# Loop over folds, fit the model, and compute the scores on training & test data
for ii in range(n_folds):
    lr = LinearRegression()
    lr.fit(X[fold_group!=ii].reshape(-1,1), y[fold_group!=ii])
    self_score[ii] = lr.score(X[fold_group!=ii].reshape(-1,1), y[fold_group!=ii])
    cross_score[ii] = lr.score(X[fold_group==ii].reshape(-1,1), y[fold_group==ii])

# Print out the results
print(f'Training Score: {self_score}')
print(f'Testing Score: {cross_score}')


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

What is the average test score across folds?

In [ ]:
# Take the mean of the scores across folds


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

`sklearn` provides a convient object for splitting data, so that you don't need to write your own splitting code. It is called `KFolds` and is housed in the `model_selection` modual.

Just as we did with the `Regression` object, the `scikit-learn` interface has us instantiate a `KFold` object, which provides a generator object that we can use to loop through our data.

In [ ]:
from sklearn.model_selection import KFold

# Generate the folds
folderizer = KFold(n_splits=5)
folderizer.split(X, y)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

What is a generator object? Each time it is called, it will generate the next of n folds in our data. It can therefore be incorporated into a for loop using the following syntax:

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Let's repeat the exercise above, to fit the model and compute the score across folds, but now using the `KFold` generator object

In [ ]:
# Initialize KFold object
n_folds = 5
folderizer = KFold(n_splits=n_folds,)

# Create an empty array to save the data in
self_score = np.empty(n_folds)
cross_score = np.empty(n_folds)
# Loop through folds, fit model, save scores
for ii, (train_index, test_index) in enumerate(folderizer.split(X, y)):
    lr = LinearRegression()
    lr.fit(X[train_index].reshape(-1,1), y[train_index])
    self_score[ii] = lr.score(X[train_index].reshape(-1,1), y[train_index])
    cross_score[ii] = lr.score(X[test_index].reshape(-1,1), y[test_index])
# Print out the results
print(f'Training Score: {self_score}')
print(f'Testing Score: {cross_score}')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Compute the average testing score across folds.

In [ ]:
# Take the mean and print it out


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Discussion questions:

(1) Why is this different from what we did before? Hint: read the documentation on KFold so see how it splits the data.

(2) What does this tell us about pupil responses in our data?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3>Question 3: How well can you predict neural activity based on stimulus information? Does the prediction improve when additional variables are included?</h3>

One of the useful things about regression models is that that can be used evaluate the role of different kinds of features in predicting data. 

Earlier, we saw that examples of how stimulus identity can be encoded by the activity of a single neuron. Here, we will recast this tuning problem as a regression problem, allowing us to use our regression tool-box to understand this tuning. 

We will then see how using a common modeling framework allows us to quantitatively compare the encoding of different features by analyzing the contribution of each feature to explaining variance in neural activity.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3>Casting tuning as a regression problem</h3>
    
    
Because our stimulus here consists of a set of 8 discrete images, we need to adapt this framework to make predictions based on predictions are based on a categorical rather than continuous variable (i.e., one of $8$ possible images). In other words, as above, we seek a model of the form:

$$y = \beta x+C,$$

where $y$ is the calcium response, $X$ is now the stimulus identity (a catigorical variable), and $\beta$ and $C$ are constants. 

One way to handle this would be to construct a separate model for each orientation:
$$y = \beta_1 X_1+C_1$$
$$y = \beta_2 X_2+C_2$$
$$\vdots$$
$$y = \beta_8 X_8+C_8 $$
    
Mathematically, this is cumbersome - we would need to look up which equation to use each time we want to predict new data. A more elegant alternative is to combine predictors across orientations into a single model that simply operates piecewise:

$$y = C+ \begin{cases} 
\beta_1 X * \text{I}_1(X)  \\
\beta_2 X * \text{I}_2(X) \\
\vdots \\
\beta_8 X * \text{I}_8(X)
\end{cases}$$
    
where $\text{I}_n(X)$ is the <i>indicator function</i>:
$$ \text{I}_n(X) := \begin{cases}
1 \text{ if } X=n, \\
0 \text{ else}
\end{cases} $$

(Notice that this formulation merges the constants into one value, $C$. $C$ is, effectivly, the offset from zero for any model we fit.)

Thus, as $X$ encodes the stimulus identity, $\text{I}_n(X)$ determines which term in the equation we are operating with. This type of problem is called *"One-Hot"* encoding, because $X$ encodes what part of the equation is active. Practically speaking, we can implement this indicator function by creating a vector for each sample and setting $X_i = 1$ for whichever case is true. For example, if we had just two stimulus types, we might have: 

$$ X_1 = [1,0] $$
$$ X_2 = [0,1] $$ 

Finally, if we have many observations, we can stack each of these $X$ observations to form a "Design Matrix." 

We will have a corresponding fitting parameter vector, $$\beta = [\beta_1,\beta_2,\ldots,\beta_8]$$

Our whole problem can now be written: 
$$y = \beta X$$ 

    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Let's build out our X matrix (the features we are using to predict our cell activity), as described above, using the stimulus identity for each stimulus presentation

In [ ]:
# Build the X matrix, which is the image identity presented on each trial, encoded as a one-hot vector

# Get index for each image for each stimulus presentation 
img_index = cell_df.image_index.values
# Create an array the length of stimulus presentations by 8 (the number of images)
X = np.zeros((len(img_index),8))
# Loop through image indices and build up the X matrix 
for ii in range(len(img_index)):
    X[ii,img_index[ii]]  = 1


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now plot it. Label the axes. 

In [ ]:
# Plot the X matrix 


Looks pretty gross, right? X is too tall a matrix to be easily visualized. Lets zoom in on the first bit of it to get a sense for whats really going on.

In [ ]:
# This is more intuitive if we zoom in
fig,ax = plt.subplots()
ax.imshow(X[:20]) 
ax.set_ylabel('Stimulus presentation #')
ax.set_xlabel('Image #')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Now that we have our design matrix, X, using using it to fit a model is quite simple. We use the same `LinearRegression` object as before, but now fit with our new design matrix.  


In [ ]:
# Set up the model with the LinearRegression object and our new design matrix X, with the cell activity y
model = LinearRegression(fit_intercept=False).fit(X, y)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now plot the model coefficients, which are the predictions of the cell's response for each image. 

How does this compare to the turning curve we plotted earlier?

In [ ]:
# Plot the coef_ attribute. Label the axes. 


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

You will have notice that we used a new setting when we created the `LinearRegression` object, `fit_intercept=False`. This prevents `LinearRegression` from fitting the constant/intercept term in our model. 

If we were to include this term, the model fit would be ill posed. Our model performance would be the same, but our data would be shifted by an aribtary constant. To see this, try fitting a model with `fit_intercept=True` and looking `coef_` and `intercept_`.

In [ ]:
# Fit the model with fit_intercept = True and look at the coefficients and intercept
funky_model = LinearRegression(fit_intercept=True).fit(X, y)
print(f'Coefs: {funky_model.coef_}')
print(f'Intercept: {funky_model.intercept_}')

In [ ]:
# Because the intercept is just a shift in the data, we can add it back to the
# coefs to recover our origional model
fig,ax = plt.subplots()
ax.plot(funky_model.coef_+funky_model.intercept_)
ax.set_xlabel('image index')
ax.set_ylabel('Model Coefficient')


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now that we know how to use our design matrix, we properly evaluate it using Kfold Cross validation. 

This is done exactly as we did above, with the addition of our new regression parameters.

In [ ]:
# First set the seed so you get the same result here no matter what order you run this notebook in!
np.random.seed(5) 

# Initialize KFold object
folderizer = KFold(n_splits=5,shuffle=True)

# Create arrays to save the results
self_score = np.empty(n_folds)
cross_score = np.empty(n_folds)
models = [None]*5

# Loop over folds, fit the model and collect the scores
for ii, (train_index,test_index) in enumerate(folderizer.split(X,y)):
    models[ii] = LinearRegression(fit_intercept=False).fit(X[train_index,:], y[train_index])
    self_score[ii] = models[ii].score(X[train_index,:], y[train_index])
    cross_score[ii] = models[ii].score(X[test_index,:], y[test_index])
print(f'Training Score: {self_score}')
print(f'Testing Score: {cross_score}')
print(f'Average Testing Score: {np.mean(cross_score)}')


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Loop through the folds, as we did above, but now plot the coefficients for each fold to check how consistent the results are

In [ ]:
# Create a plot axis to visualize the results
fig,ax = plt.subplots()

# Loop over folds and plot the coefficients


# Label the axes


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> How do stimulus and behavior information compare? </h3>
    
By fitting models with different combinations of features, we can get a richer sense of how different features are encoded by neural activity. 

One option for doing this is simply to fit a model to each variable of interest and compare their performance. This answers a very simple question: how much of a cells variability can be explained by this particular feature. We will see, however, that when variables are correlated the outcome of such one-at-a-time model fits can be difficult to interpret. 


This next step is going to involve a bunch of model fitting, using the same basic procedure we outlined in the previous section. Before we go on, lets take a quick momement to move our KFold Linear Model fitting into a function so we don't have to type so much!

Note that the `KFold` object includes both `shuffle` and `shuffle_seed` parameters. `shuffle` does exactly what it sounds like- it randomizes the set data points included in each fold. `shuffle_seed` can be used to get reproducible results from this shuffling. This is particularly important if we want to compare models- using the same shuffle seed will give the same random set of trials across function calls.

In [ ]:
def crossValidateLinearModel(X, y , n_split = 5, shuffle = False, shuffle_seed = None):
    '''
    Cross validate a linear model using KFold cross validation

    Parameters
    ----------
    X : np.array
        The input data to fit
    y : np.array
        The output data to fit
    n_split : int
        The number of splits to use
    shuffle : bool
        Whether or not to shuffle the data
    shuffle_seed : int
        The seed to use for shuffling the data

    Returns
    -------
    Mean Score: float
        The average cross validation score
    Model List: list    
        The models fit to each fold of the data
    Test score: np.array    
        The cross validation scores for testing data each fold
    Train score: np.array
        The cross validation scores for testing data each fold
    '''

    if len(X.shape)==1:
        X = X.copy().reshape(-1,1)
    # Initialize KFold object
    folderizer = KFold(n_splits=n_split,shuffle=shuffle,random_state=shuffle_seed)
    # Create an array to save the results
    self_score = np.empty(n_folds)
    cross_score = np.empty(n_folds)
    models = [None]*n_split
    # Loop through the folds, fit the model, and save the results
    for ii, (train_index, test_index) in enumerate(folderizer.split(X,y)):
        models[ii] = LinearRegression(fit_intercept=False).fit(X[train_index,:], y[train_index])
        self_score[ii] = models[ii].score(X[train_index,:], y[train_index])
        cross_score[ii] = models[ii].score(X[test_index,:], y[test_index])
        
    return np.mean(cross_score),models,cross_score,self_score

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Create an X matrix for each variable we want to test: stimulus, pupil, and running. 

Use the `mean_pupil_width`, `mean_running_speed` and `mean_response` columns of the `stimulus_response_df` for our cell. 

Remember that we previously we saved this data as `cell_df` above.

Print out the shapes of each X matrix. 

Plot the X matrices for tunning and pupil to see what they look like. 

In [ ]:
# Create design matrix for each feature
X_stim = X.copy()
X_pupil = cell_df.mean_pupil_width.values
X_running = cell_df.mean_running_speed.values

In [ ]:
# What are their shapes?


In [ ]:
# Plot the running and pupil design matrices to see what they look like
# Bonus if you use a twinx() to see both on the same axis




<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the function we created above to cross-validate and test our linear model for each of the variables. 

Which one produces the highest score? 

In [ ]:
# Now use our fancy new function to test a bunch of models.
seed = 5
x_stim_score,_,_,_  = crossValidateLinearModel(X_stim, y, shuffle = True, shuffle_seed=seed)
print(f'Stimulus model score {x_stim_score}')

x_pupil_score,_,_,_  = crossValidateLinearModel(X_pupil, y, shuffle = True, shuffle_seed=seed)
print(f'Pupil model score {x_pupil_score}')

x_running_score,_,_,_  = crossValidateLinearModel(X_running, y, shuffle = True, shuffle_seed=seed)
print(f'Running model score {x_running_score}')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h4> What if our variables are correlated? </h4>

At face value it looks like our cell of choice heavily encodes stimulus identify, with weaker encoding of running and pupil size.

So...can we go to an early lunch? Not quite. The challence here is that running and pupil diameter are not necessarily indpendent variables. This makes that fact that our neuron shows a weak correlation with both of them difficult to interpret.

Let's plot the relationship of running and pupil

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now, lets return to the relationship between <b>pupil width</b> and <b>running</b> , using our design matrices which contain the average value of these variables for each stimulus presentation. 

Compute the <b>pearson correlation</b>  and put it in the title of the plot. Is the relationship significant?

In [ ]:
# Plot pupil vs running. Label your axes.
fig, ax = plt.subplots()
ax.scatter(X_pupil, X_running)
ax.set_xlabel('Mean pupil')
ax.set_ylabel('Mean running')

# Compute the correlation and print it out
r, p = pearsonr(X_pupil, X_running)
ax.set_title('r = '+str(np.round(r, 3)))
print(p)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Question 4: Does the prediction improve when additional variables are included? </h3>

In regression models, we are not limited to considering each feature one-at-a time. Instead, multiple features or sets of features can be combined into a single model simply by combining them in the design matrix used to train that model.

This is particularly useful when trying to determine if correlated features are uniquely encoded by a cell. To close today, we will see two methods for asking whether or not a given feature explains some of cell's variance beyond what could have been explained by our other modeled features. 


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Multiple linear regression </h3>

*Multiple Linear Regression* gives us tools to dissect the contributions of different features in explaining variance.

Just as we built a design matrix out of different stimulus identities, we can similary construct one that includes additional features about our data.

Let's create an X matrix that incorporates stimulus, pupil, and running

In [ ]:
# Stack up the x matrices to make one big feature matrix
X_combo = np.hstack((X_stim, X_pupil.reshape(-1,1), X_running.reshape(-1,1)))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use our `crossValidateLinearModel` function from before to get the model prediction for this multi-variate X matrix.

In [ ]:
# Provide X_combo and y to the function we defined above
x_combo_score,_,_,_  = crossValidateLinearModel(X_combo, y, shuffle=True, shuffle_seed=seed)
print(f'Combined model score {x_combo_score}')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

Importantly, we cannot simply look at the model coefficients, as we did in the "stimulus only" example. This is because our model now contains different types of features with different magnitudes, and there is not a clear mapping between them. While the weights we fit will scale accordingly, they can no longer be directly compared. Visualizing the design matrix illustrates this problem.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Plot the design matrix `X_combo` for the first 20 stimulus presentations.

In [ ]:
fig,ax = plt.subplots()
ax.imshow(X_combo[:20]) 
ax.set_ylabel('Trial #')
ax.set_xlabel('Image #')

This shows images 1-8, then running speed for each trial, then pupil width for each trial

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Leave one out test </h4>

Instead, we can use the model scores - that is, the variance in our data explained by our model - to test the encoding of any particular feature.

Specifically, we can systematically drop out feature one at a time and see how model performance changes. If the model gets worse, it suggests that this feature was explaining some of the variance in our data. Because other features are still included, this method is a way to avoid mistakenly assuming that a cell encodes all of a set of correlated variables.

Let's create several design matrices, each with one of the variables left out

In [ ]:
# Create each design matrix as a stacked combo of all the features except the one we are leaving out
X_wout_stimulus = np.hstack((X_pupil.reshape(-1,1), X_running.reshape(-1,1)))
X_wout_running =  np.hstack((X_stim, X_pupil.reshape(-1,1)))
X_wout_pupil =  np.hstack((X_stim, X_running.reshape(-1,1)))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Use the `crossValidateLinearModel` function again to test model performance on each of the leave one out design matrices.

In [ ]:
# Now use our fancy new function to test a bunch of models.
x_combo_score,_,_,_  = crossValidateLinearModel(X_combo, y, shuffle = True, shuffle_seed=seed)
print(f'Combo {x_combo_score}')

x_wout_stim_score,_,_,_  = crossValidateLinearModel(X_wout_stimulus, y, shuffle = True, shuffle_seed=seed)
print(f'Wout Stim {x_wout_stim_score}')
print(f'Additional variance explained by stim {x_combo_score-x_wout_stim_score}')

x_wout_pupil_score,_,_,_  = crossValidateLinearModel(X_wout_pupil, y, shuffle = True, shuffle_seed=seed)
print(f'Wout Pupil {x_wout_pupil_score}')
print(f'Additional variance explained by pupil {x_combo_score-x_wout_pupil_score}')

x_wout_running_score,_,_,_  = crossValidateLinearModel(X_wout_running, y, shuffle = True, shuffle_seed=seed)
print(f'Wout_Running {x_wout_running_score}')
print(f'Additional variance explained by running {x_combo_score-x_wout_running_score}');


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

<h4> Impact of behavioral choice on model prediction </h4>

In part 2 above, we also suggested that cells encoded the difference between <b>hit</b> and <b>miss</b> trials (image changes with or without a lick + reward). 

Build a complete model that includes these features using the same one-hot encoding method we used for the stimulus, then use this to quanitfy the relative contribution of stimulus identity vs. choice

In [ ]:
# Get values from the stimulus response df for our cell
X_hit = cell_df.hit.values
X_lick = cell_df.licked.values

In [ ]:
# Construct the full model
X_full = np.hstack((X_stim, X_pupil.reshape(-1,1), X_running.reshape(-1,1), X_hit.reshape(-1,1), X_lick.reshape(-1,1)))

# And subset models lacking each variable
X_wout_hit = np.hstack((X_stim, X_pupil.reshape(-1,1), X_running.reshape(-1,1), X_lick.reshape(-1,1)))
X_wout_lick = np.hstack((X_stim, X_pupil.reshape(-1,1), X_running.reshape(-1,1), X_hit.reshape(-1,1)))

In [ ]:
# Evaluate
x_full_score,_,_,_  = crossValidateLinearModel(X_full, y, shuffle = True,shuffle_seed=seed)
print(f'Combo {x_full_score}')

x_wout_hit_score,_,_,_  = crossValidateLinearModel(X_wout_hit, y, shuffle = True,shuffle_seed=seed)
print(f'Wout hits {x_wout_hit_score}')
print(f'Additional variance explained by hit-vs-miss {x_full_score-x_wout_hit_score}')

x_wout_lick_score,_,_,_  = crossValidateLinearModel(X_wout_lick, y, shuffle = True, shuffle_seed=seed)
print(f'Wout licks {x_wout_lick_score}')
print(f'Additional variance explained by licking {x_full_score-x_wout_lick_score}')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

This result may be surpising, given that previously we showed that tunning can be different on hits vs misses.

To understand why this is happening, we need to think about what question we are asking with our model. In the previous problem, we used a single feature to encode whether a trial was a hit or a miss. This allows for changes in overal response on hit vs miss trials, but cannot account more subtil difference in tuning.

There are a couple ways to answer this question. One would be to adapt the linear modeling framework to handle two conditions of stimuli: hits vs misses

In [ ]:
# Find which trials are hits and misses
is_hit = X_hit.astype(bool)

# Build two copies of the stimulus matrix: one for hits and one for misses
# When a trial is not in that category, set all values to 0.
X_hit_stim = X_stim.copy()
X_hit_stim[~is_hit,:] = 0
X_miss_stim = X_stim.copy()
X_miss_stim[is_hit,:] = 0

# Combine everything into a single design matrix.
X_seperate_stim = np.hstack((X_hit_stim, X_miss_stim))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now fit the model!

In [ ]:
# Use the LinearRegression object with X_separate_stim matrix and the y value (cell responses) to predict
lr = LinearRegression(fit_intercept=False).fit(X_seperate_stim,y)


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Now when we will, effectively, get two sets of coefficient: one for hit trials and one for miss trials. They will be combined into a single coefficient vector. 

Plot each set of coefficients; do you see differences? 

In [ ]:
# Plot the coefficients for the hits and misses separately
fig,ax = plt.subplots()
ax.plot(lr.coef_[:8],label= 'Hits')
ax.plot(lr.coef_[8:],label = 'Misses')
ax.set_xlabel('Image index')
ax.set_ylabel('Model coef')
ax.legend()


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

In this case, simply using random cross validation would be insufficient. This is because we are somewhat data-starved in this problem: There are not equal numbers of hits and misses, and they are not randomly distributed throughout the session.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Look at the pattern of hits and misses across trials. How many are there of each? When do they happen during the session? 

In [ ]:
# plot X_hit over time to see how many trials are hits vs misses


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

For now, we can return to using the `train_test_split` function to split our data in half. Due to the nature of our questions, however, we are going to get to try out a few extra features of this function.

First, We will `stratify` our data using the 'hit' variable. This will ensure that both halves of the data have equal numbers of hits and misses.

Next, we will force the training and test sets to be of equal size, each containing half of our data.

We will call this function twice, once to split the design matrix with our trials separated, once for the un-separated data. Because we want to get the same trial splits between these two conditions, we will set `random_state` to be the same between these function calls.

In [ ]:
# Setting random_state to be the same here ensure that we grab  the same halves of both design matricies.
y_train, y_test, X_sep_train, X_sep_test = train_test_split(y, X_seperate_stim, stratify=X_hit, test_size=.5, train_size=.5, random_state=0)
_, _, X_nosep_train, X_nosep_test = train_test_split(y, X_stim, stratify=X_hit,test_size=.5, train_size=.5, random_state=0)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

Fit two models with the training data we just pulled out: one using separated stimuli and the other using un-separated stimuli. Evaluate these models using the test data. How much of a difference does splitting hits and miss stimuli make?

In [ ]:
lr_sep = LinearRegression(fit_intercept=False).fit(X_sep_train, y_train)
print(f'Mean variance explained with hits and misses seperated {lr_sep.score(X_sep_test, y_test)}')
lr_nosep = LinearRegression(fit_intercept=False).fit(X_nosep_train, y_train)
print(f'Mean variance explained with without seperation {lr_nosep.score(X_nosep_test, y_test)}')
print(f'Improvment from seperation {lr_sep.score(X_sep_test, y_test)-lr_nosep.score(X_nosep_test, y_test)}')


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>


The other way to aproach this problem would be using the predictive nature of the linear model. We can train a model on one kind of data (e.g misses) and ask how well it does at explaining misses. Because there are far more "miss" trials, lets use split these to train our model

We can start by seperating 'hit' and 'miss' trials

In [ ]:
X_stim_miss = X_stim[~is_hit,:]
y_miss = y[~is_hit]
X_stim_hit = X_stim[is_hit,:]
y_hit = y[is_hit]

Now, lets build a model using only miss trials. 

We still need to separate the miss trials in training and testing data, so that we can compare our model performance on left out miss trials to its performance on hit trials

In [ ]:
# Split miss trials
y_miss_train, y_miss_test, X_stim_miss_train, X_stim_miss_test = train_test_split(y_miss, X_stim_miss, test_size=.5, train_size=.5, random_state=0)
# Fit model
lr_miss = LinearRegression(fit_intercept=False).fit(X_stim_miss_train, y_miss_train)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>

We can now compare the performance of a 'miss' trial model in predicting both held-out miss trials and hit trials. 

Compute the score for the miss model with the test data with misses, compared to the hit trials. Is a miss trial model any good at predicting activity durring hit trials?

In [ ]:
# Performance on misses test data
lr_miss.score(X_stim_miss_test, y_miss_test)

In [ ]:
# Performance on hits
lr_miss.score(X_stim_hit, y_hit)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
<h3> Question 5: Going nonlinear [BONUS MATERIAL] </h3>


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>

So far se have focused on linear models of the form 

$f(X) = \beta X = \beta_0X_0 + \beta_1X_1+...+\beta_nXn$

This is commonly refered to as a "General Linear Model," or gLM. These are super useful, but this model class makes a big assumption about the variance distribution of our data: specifically, we assumed that variance is normally distributed about some mean value, $f(x)$.

We can, however, arbitrarly modify this variance assumption. To achieve this this, we a non-linearly, $g(*)$, to our equation:

$f(X) = g(\beta X)$

or equlivalently:

$g(f(X))^{-1}= \beta X$

This later case is what you will see in literature more often, where $g(*)^{-1}$ is commonly called the 'link' function. 

In principle, $g(*)$ can be whatever we want. We choose $g(*)^{-1}$ that match the variance model we want to capture. Adding the nonlinearity, however, ruins the closed-form solution for $\beta$ that we elluded to in the strictly linear case. Instead, we need to use an optimizer to solve for $\beta$ in this new non-linear case. Such solvers work by maximizing the likelyhood (or equivalently minimizign the negative log-likelihood) of potential $\beta$ values for a given dataset. If $g(*)$ describes variance models from the exponential family of distributions, it can be shown that convergence is garanteed for this optimization problem.

The class of models created by including $g(*)$ is commonly refered to as "General*ized* Linear Models," or GLMs. Yes, this nomenclature is confusing- if it makes you feel any better, the people who created it later admitted they should have chosen a better name. You will sometimes also hear of these models refered to as "Linear-Non-linear" models, because the linear and nonlinear parts can be mathematically seperated.


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Why do we bring up these these non-linearities? 
    
There is a specific case that frequently arrising in neuroscience where the assumption of normally distributed variance is particularly bad: *spiketrains*.

Spikes are discrete, transient events. Over a given time interval, there can never be fewer than 0 spikes, and there can never be a fractional number of spikes. If we assume that spikes are occure indepently with an average rate (call it $\lambda$), then a poisson distribution (https://en.wikipedia.org/wiki/Poisson_distribution) can be shown to be a good model of this variance. To give our GLM poisson varinace, we use the log link function $g(\lambda)^{-1}=ln(\lambda)$.

Now, our goal with our GLM will be to predict what the mean rate, so we need to fit a model of the fit our new model:

$ln(\lambda) = \beta X$
    
Fortunatly, `sklearn` ships with an implementation of poisson regression, as well as many other commonly used link functions (e.g. those for exponential data, logistic data, etc. https://scikit-learn.org/stable/modules/linear_model.html#generalized-linear-models). 
    
In practice, this means that switching to a Poisson GLM requires relativly minimal changes to our code.



In [ ]:
from sklearn.linear_model import PoissonRegressor

PLM = PoissonRegressor(fit_intercept=False)
PLM.fit(X_stim,y)

Right now we aren't working with spikes! The sklearn package won't accept non discrete data as input, since poisson regression assumes that data must be integer values. 

In the next workshop you will delve more deeply into spiking data. Later, as an excersize, come back to this section and try some of the regression techniques we saw above with spiketrains and poisson regression. This generalized cross validation function should help get you started:

In [ ]:
def crossValidateGeneralModel(X,y,model =LinearRegression,n_split = 5,shuffle = False,shuffle_seed = None):
    '''
    Super informative docstring goes here! 
    Should writing it be an excersize?
    '''
    if len(X.shape)==1:
        X = X.copy().reshape(-1,1)
    folderizer = KFold(n_splits=n_split,shuffle=shuffle,random_state=shuffle_seed)
    self_score = np.empty(n_folds)
    cross_score = np.empty(n_folds)
    models = [None]*n_split
    for ii, (train_index,test_index) in enumerate(folderizer.split(X,y)):
        models[ii] = model(fit_intercept=False).fit(X[train_index,:],y[train_index])
        self_score[ii] = models[ii].score(X[train_index,:],y[train_index])
        cross_score[ii] = models[ii].score(X[test_index,:],y[test_index])
    return np.mean(cross_score),models,cross_score,self_score

Note that the `PoissonRegressor.score` function behaves slightly differently from the linear regression version. It reterns "Deviance explained," rather than "Variance Explained." The difference are subtil, but means that the two numbers cannot be directly compared. Read more about these metrics here https://scikit-learn.org/stable/modules/model_evaluation.html#d2-score